# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
import sys
sys.path.append('C:/Users/dark_/Documents/GitHub/python-api-challenge/')
from api_keys import geoapify_key

In [27]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,invercargill,-46.400,168.3500,11.73,91,65,2.42,NZ,1683590288
1,1,adamstown,-25.066,-130.1015,21.53,56,100,5.46,PN,1683590293
2,2,yasothon,15.750,104.1333,29.46,69,100,2.73,TH,1683590299
3,3,port-aux-francais,-49.350,70.2167,6.34,95,100,19.75,TF,1683590305
4,4,broken hill,-31.950,141.4333,11.70,63,1,6.41,AU,1683590310


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [26]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True,
                                      color = 'City',
                                      alpha = 0.6,
                                      size = "Humidity",
                                      tiles = "OSM",
                                      width=1000, height=550
                                     )
map_plot
# Display the map   
#city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [36]:
# Narrow down cities that fit criteria and drop any results with null values
# Using the suggested filters: A max temperature lower than 27 degrees but higher than 21, Wind speed less than 4.5 m/s & Zero cloudiness
filtered_cities_df = city_data_df[(city_data_df['Cloudiness'] == 0) & (city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] > 21) & (city_data_df['Wind Speed'] < 4.5)]

# Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Display sample data
filtered_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
115,115,lebanon,36.2081,-86.2911,26.12,83,0,2.06,US,1683590719
213,213,saint-pierre,-21.3393,55.4781,22.82,73,0,4.12,RE,1683591607
253,253,yanbu,24.0891,38.0637,23.02,68,0,4.12,SA,1683591730
259,259,belmonte,-15.8631,-38.8828,24.53,87,0,3.61,BR,1683591870
264,264,morondava,-20.2833,44.2833,23.10,73,0,3.03,MG,1683591898
271,271,chillicothe,39.3331,-82.9824,22.17,68,0,0.00,US,1683591937
282,282,lebrija,36.9208,-6.0753,23.22,72,0,0.51,ES,1683592009
288,288,les avirons,-21.2418,55.3394,22.03,73,0,4.12,RE,1683592042
315,315,ha'il,27.5219,41.6907,22.01,28,0,1.03,SA,1683592091
325,325,prado,-17.3411,-39.2208,23.71,89,0,2.89,BR,1683592270


### Step 3: Create a new DataFrame called `hotel_df`.

In [54]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities_df[['City', 'Country', 'Lat', 'Lng','Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

C:\Users\dark_\AppData\Local\Temp\ipykernel_6216\2209748293.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ''


,City,Country,Lat,Lng,Humidity,Hotel Name
115,lebanon,US,36.2081,-86.2911,83,
213,saint-pierre,RE,-21.3393,55.4781,73,
253,yanbu,SA,24.0891,38.0637,68,
259,belmonte,BR,-15.8631,-38.8828,87,
264,morondava,MG,-20.2833,44.2833,73,
271,chillicothe,US,39.3331,-82.9824,68,
282,lebrija,ES,36.9208,-6.0753,72,
288,les avirons,RE,-21.2418,55.3394,73,
315,ha'il,SA,27.5219,41.6907,28,
325,prado,BR,-17.3411,-39.2208,89,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [55]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories': 'accommodation.hotel',
    'apiKey':geoapify_key
    }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    
# Display sample data
hotel_df

Starting hotel search
lebanon - nearest hotel: Ramada
saint-pierre - nearest hotel: Tropic Hotel
yanbu - nearest hotel: No hotel found
belmonte - nearest hotel: No hotel found
morondava - nearest hotel: Hotel Sharon Menabe
chillicothe - nearest hotel: Quality Inn Chillicothe
lebrija - nearest hotel: Hotel LB
les avirons - nearest hotel: Floralys
ha'il - nearest hotel: فندق كلاسيك
prado - nearest hotel: Pousada Recanto do Prado
hell-ville - nearest hotel: ABUD HOTEL
tamanrasset - nearest hotel: فندق الأمان
kearney - nearest hotel: Days Inn
akalgarh - nearest hotel: No hotel found
hobe sound - nearest hotel: No hotel found
tanhacu - nearest hotel: Tanhaçu Palace
bay wood - nearest hotel: Bay Shore Inn
timimoun - nearest hotel: أناف


,City,Country,Lat,Lng,Humidity,Hotel Name
115,lebanon,US,36.2081,-86.2911,83,Ramada
213,saint-pierre,RE,-21.3393,55.4781,73,Tropic Hotel
253,yanbu,SA,24.0891,38.0637,68,No hotel found
259,belmonte,BR,-15.8631,-38.8828,87,No hotel found
264,morondava,MG,-20.2833,44.2833,73,Hotel Sharon Menabe
271,chillicothe,US,39.3331,-82.9824,68,Quality Inn Chillicothe
282,lebrija,ES,36.9208,-6.0753,72,Hotel LB
288,les avirons,RE,-21.2418,55.3394,73,Floralys
315,ha'il,SA,27.5219,41.6907,28,فندق كلاسيك
325,prado,BR,-17.3411,-39.2208,89,Pousada Recanto do Prado


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [56]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.6,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"],
                                   width=1000, height=550
                                   )

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)